# XOR problem

## Import

In [ ]:
import tensorflow as tf
import FFBP

## Build

In [ ]:
USE_SAVED_PARAMS = True # set to True for the first part of the exercise; use False for part two

WR = .5  # weight initialization range
LR = .25 # learninig rate
M  = .9  # momentum rate

FFBP_GRAPH = tf.Graph()

with FFBP_GRAPH.as_default():
    
    with tf.name_scope('train_data'):
        train_data = FFBP.InputData(
            path_to_data_file = 'xor_data.txt',
            inp_size = 2, 
            targ_size = 1,
            batch_size = 4, 
            data_len = 4,
            shuffle_seed = None
        )

    with tf.name_scope('test_data'):
        test_data = FFBP.InputData(
            path_to_data_file = 'xor_data.txt',
            inp_size = 2, 
            targ_size = 1,
            batch_size = 1,
            data_len = 4,
        )

    # NETWORK CONSTRUCTION
    model_name = 'xor_model'
    with tf.name_scope(model_name):

        input_  = tf.placeholder(dtype = tf.float32, shape=[None, 2], name='model_inp')

        hidden_layer = FFBP.BasicLayer(
            layer_name = 'hidden_layer', 
            layer_input = input_, 
            size = 2, 
            wrange = [-WR, WR], 
            nonlin = tf.nn.sigmoid, 
            seed = -1
        )

        output_layer = FFBP.BasicLayer(
            layer_name = 'output_layer', 
            layer_input = hidden_layer.output, 
            size = 1, 
            wrange = [-WR, WR], 
            nonlin = tf.nn.sigmoid, 
            seed = -1
        )

        target = tf.placeholder(dtype = tf.float32, shape=[None, 1], name='targets')

        MODEL = FFBP.Model(
            name = model_name,
            layers = [hidden_layer, output_layer],
            train_data = train_data, 
            inp        = input_,
            targ       = target,
            loss       = tf.reduce_sum(tf.squared_difference(target, output_layer.output), name='loss_function'),
            optimizer  = tf.train.MomentumOptimizer(LR, M),
            test_data  = test_data
        )

## Run

In [ ]:
# Set up run parameters
if USE_SAVED_PARAMS:
    NUM_RUNS = 1
    NUM_EPOCHS = 1000
    TEST_EPOCHS = [i for i in range(0, NUM_EPOCHS, 30)]
    SAVE_EPOCHS = [None]
    ECRIT = .01
    RESTORE_DIR = 'exercise_params' # path to logdir with 'checkpoint_files' directory to restore a saved model
else:
    NUM_RUNS = 3
    NUM_EPOCHS = 500
    TEST_EPOCHS = [i for i in range(0, NUM_EPOCHS, 50)]
    SAVE_EPOCHS = [None]
    ECRIT = 0
    RESTORE_DIR = None

# Create ModelSaver to manage test data saving and model checkpointing:
saver = FFBP.ModelSaver(restore_from=RESTORE_DIR, logdir=None)

for run_ind in range(NUM_RUNS):
    print('>>> RUN {}'.format(run_ind))
    
    with tf.Session(graph=FFBP_GRAPH) as sess:

        # restore or initialize FFBP_GRAPH variables:
        start_epoch = saver.init_model(session=sess)

        # create coordinator and start queue runners
        coordinator = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coordinator)

        for i in FFBP.prog_bar(
            sequence=range(start_epoch, start_epoch + NUM_EPOCHS), 
            name='Run {}/{}, Epoch'.format(run_ind+1, NUM_RUNS)):
            
            # Test model occasionally
            if any([i==test_epoch for test_epoch in TEST_EPOCHS]):
                testloss, snap = MODEL.test_epoch(session=sess, verbose=True)
                saver.log_test(snap, run_ind)

            # Run one training epoch
            loss, enum = MODEL.train_epoch(session=sess, verbose=False)
            saver.log_loss(loss, enum, run_ind)

            # Save model occasionally
            if any([i==save_epoch for save_epoch in SAVE_EPOCHS]):
                checkpoint_path = saver.save_model(session=sess, model=MODEL, run_ind=run_ind)

            # Do final test, stop queues, and break out from training loop
            if loss < ECRIT or i == start_epoch + (NUM_EPOCHS - 1): 
                print('Final test ({})'.format(
                    'loss < ecrit' if loss < ECRIT else 'num_epochs reached'))

                testloss, snap = MODEL.test_epoch(session=sess, verbose=True)
                saver.log_test(snap, run_ind)

                coordinator.request_stop()
                coordinator.join(threads)

                saver.save_model(session=sess, model=MODEL, run_ind=run_ind)
                break
                
FFBP.view_progress(logdir=saver.logdir)